In [ ]:
!git clone -q https://github.com/sudo-carson/TensorFlowTTS.git
!pip install -q git+https://github.com/repodiac/german_transliterate /content/TensorFlowTTS
!pip install keras==2.6.0

In [ ]:
import tensorflow as tf

In [ ]:
!pip install onnxmltools
!pip install onnx
!pip install tf2onnx
# !pip install onnxruntime # for CPU
!pip install onnxruntime-gpu # for GPU

In [ ]:
import numpy as np
import soundfile as sf
import yaml
import IPython.display as ipd

import tensorflow as tf

from tensorflow_tts.inference import AutoProcessor
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoConfig


mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")
processor = AutoProcessor.from_pretrained(
    pretrained_path="processor.json")
config = AutoConfig.from_pretrained("config.yml")
fastspeech2 = TFAutoModel.from_pretrained(
    config=config, 
    pretrained_path="model.h5"
)

In [ ]:
import tf2onnx
import onnxruntime as rt
print(rt.get_device())

# https://github.com/onnx/tensorflow-onnx/blob/main/tutorials/keras-resnet50.ipynb

spec = (tf.TensorSpec((1, None), tf.int32, name="input_ids"), 
        tf.TensorSpec((1), tf.int32, name="speaker_ids"),
        tf.TensorSpec((1), tf.float32, name="speed_ratios"),
        tf.TensorSpec((1), tf.float32, name="f0_ratios"),
        tf.TensorSpec((1), tf.float32, name='energy_ratios'))

# output_path = "onnx_model/fast2speech.onnx" # for CPU
output_path = "onnx_model/fast2speech.cuda.onnx" # for GPU

model_proto, _ = tf2onnx.convert.from_keras(fastspeech2, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]